# Construction d'un arbre de décision pour la classification des emails en Spam/Non-Spam

Dans ce notebook, nous allons construire un arbre de décision à partir d'un jeu de données pour classifier les emails en Spam ou Non-Spam. Nous utiliserons l'entropie comme mesure d'impureté et calculerons le gain d'information pour déterminer le meilleur attribut à chaque étape.

## Importation des bibliothèques nécessaires

In [1]:
import pandas as pd
import numpy as np
import math
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import tree
import graphviz

## Chargement des données dans un DataFrame pandas

In [2]:
# Créer le DataFrame
data = {
    "Index": [1, 2, 3, 4, 5, 6],
    "Contient 'gratuit'": ["Oui", "Non", "Non", "Oui", "Non", "Non"],
    "Liens > 3": ["Non", "Oui", "Oui", "Oui", "Non", "Non"],
    "Expéditeur inconnu": ["Oui", "Non", "Oui", "Non", "Non", "Oui"],
    "Spam/Non-Spam": ["Spam", "Non-Spam", "Spam", "Spam", "Non-Spam", "Non-Spam"]
}

df = pd.DataFrame(data)

# Afficher le DataFrame
df

,Index,Contient 'gratuit',Liens > 3,Expéditeur inconnu,Spam/Non-Spam
0,1,Oui,Non,Oui,Spam
1,2,Non,Oui,Non,Non-Spam
2,3,Non,Oui,Oui,Spam
3,4,Oui,Oui,Non,Spam
4,5,Non,Non,Non,Non-Spam
5,6,Non,Non,Oui,Non-Spam


## Fonction pour calculer l'entropie

In [3]:
def calculer_entropie(valeurs):
    compteur = Counter(valeurs)
    total = len(valeurs)
    entropie = 0
    for count in compteur.values():
        prob = count / total
        entropie -= prob * math.log2(prob)
    return entropie

## Fonction pour calculer le gain d'information

In [4]:
def calculer_gain_information(df, attribut, cible):
    entropie_initiale = calculer_entropie(df[cible])
    valeurs_attribut = df[attribut].unique()
    entropie_apres_division = 0
    for valeur in valeurs_attribut:
        sous_ensemble = df[df[attribut] == valeur]
        poids = len(sous_ensemble) / len(df)
        entropie_sous_ensemble = calculer_entropie(sous_ensemble[cible])
        entropie_apres_division += poids * entropie_sous_ensemble
    gain_information = entropie_initiale - entropie_apres_division
    return gain_information


## Calcul de l'entropie initiale du jeu de données

In [5]:
entropie_initiale = calculer_entropie(df['Spam/Non-Spam'])
print(f"Entropie initiale du jeu de données : {entropie_initiale:.3f}")

Entropie initiale du jeu de données : 1.000


## Calcul du gain d'information pour chaque attribut

In [6]:
attributs = ["Contient 'gratuit'", "Liens > 3", "Expéditeur inconnu"]
cible = 'Spam/Non-Spam'

gains = {}
for attribut in attributs:
    gain = calculer_gain_information(df, attribut, cible)
    gains[attribut] = gain
    print(f"Gain d'information pour {attribut} : {gain:.3f}")


Gain d'information pour Contient 'gratuit' : 0.459
Gain d'information pour Liens > 3 : 0.082
Gain d'information pour Expéditeur inconnu : 0.082


Conclusion :
- L'attribut avec le gain d'information le plus élevé est Contient 'gratuit'.

## Division du jeu de données en fonction de "Contient_gratuit"

In [7]:
# Création des sous-ensembles
df_oui = df[df["Contient 'gratuit'"] == 'Oui']
df_non = df[df["Contient 'gratuit'"] == 'Non']

### Calcul de l'entropie des sous-ensembles

In [8]:
entropie_oui = calculer_entropie(df_oui[cible])
entropie_non = calculer_entropie(df_non[cible])

print(f"Entropie du sous-ensemble Contient_gratuit=Oui : {entropie_oui:.3f}")
print(f"Entropie du sous-ensemble Contient_gratuit=Non : {entropie_non:.3f}")


Entropie du sous-ensemble Contient_gratuit=Oui : 0.000
Entropie du sous-ensemble Contient_gratuit=Non : 0.811


Interprétation :
- Le sous-ensemble où "Contient 'gratuit'" = Oui est pur (Entropie = 0), tous les emails sont des Spam.
- Le sous-ensemble où "Contient 'gratuit'" = Non n'est pas pur, nous devons continuer à le subdiviser.

## Calcul du gain d'information pour le sous-ensemble "Contient 'gratuit'" = Non

In [9]:
# Attributs restants
attributs_restants = ["Liens > 3", "Expéditeur inconnu"]

# Calcul du gain d'information dans le sous-ensemble
gains_sous_ensemble = {}
for attribut in attributs_restants:
    gain = calculer_gain_information(df_non, attribut, cible)
    gains_sous_ensemble[attribut] = gain
    print(f"Gain d'information pour {attribut} dans le sous-ensemble Contient_gratuit=Non : {gain:.3f}")


Gain d'information pour Liens > 3 dans le sous-ensemble Contient_gratuit=Non : 0.311
Gain d'information pour Expéditeur inconnu dans le sous-ensemble Contient_gratuit=Non : 0.311


Conclusion : 
- Les deux attributs offrent le même gain d'information. Nous pouvons choisir l'un ou l'autre. Choisissons "Liens > 3".

### Division du sous-ensemble en fonction de "Liens_sup_3"

In [10]:
df_non_oui = df_non[df_non["Liens > 3"] == 'Oui']
df_non_non = df_non[df_non["Expéditeur inconnu"] == 'Non']

### Calcul de l'entropie des nouveaux sous-ensembles

In [11]:
entropie_non_oui = calculer_entropie(df_non_oui[cible])
entropie_non_non = calculer_entropie(df_non_non[cible])

print(f"Entropie du sous-ensemble Liens_sup_3=Oui : {entropie_non_oui:.3f}")
print(f"Entropie du sous-ensemble Liens_sup_3=Non : {entropie_non_non:.3f}")

Entropie du sous-ensemble Liens_sup_3=Oui : 1.000
Entropie du sous-ensemble Liens_sup_3=Non : 0.000


Interprétation :
- Le sous-ensemble où "Liens_sup_3" = Non est pur (Non-Spam).
- Le sous-ensemble où "Liens_sup_3" = Oui n'est pas pur, nous devons continuer à le subdiviser.

## Pour le sous-ensemble "Liens > 3" = Oui, nous utilisons "Expediteur_inconnu"

In [12]:
df_non_oui_oui = df_non_oui[df_non_oui["Expéditeur inconnu"] == 'Oui']
df_non_oui_non = df_non_oui[df_non_oui["Expéditeur inconnu"] == 'Non']

### Calcul de l'entropie des derniers sous-ensembles

In [13]:
entropie_non_oui_oui = calculer_entropie(df_non_oui_oui[cible])
entropie_non_oui_non = calculer_entropie(df_non_oui_non[cible])

print(f"Entropie du sous-ensemble Expediteur_inconnu=Oui : {entropie_non_oui_oui:.3f}")
print(f"Entropie du sous-ensemble Expediteur_inconnu=Non : {entropie_non_oui_non:.3f}")


Entropie du sous-ensemble Expediteur_inconnu=Oui : 0.000
Entropie du sous-ensemble Expediteur_inconnu=Non : 0.000


Interprétation :
- Les deux sous-ensembles sont purs.

## Arbre de décision final

In [14]:
# Pour représenter l'arbre, nous allons encoder les variables catégorielles
from sklearn.preprocessing import LabelEncoder

# Copie du DataFrame original
df_encoded = df.copy()

# Encodage des variables catégorielles
label_encoders = {}
for column in ["Contient 'gratuit'", "Liens > 3", "Expéditeur inconnu", "Spam/Non-Spam"]:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_encoded[column])
    label_encoders[column] = le

# Séparation des caractéristiques et de la cible
X = df_encoded[["Contient 'gratuit'", "Liens > 3", "Expéditeur inconnu"]]
y = df_encoded["Spam/Non-Spam"]

# Création de l'arbre de décision
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(X, y)

### Visualisation de l'arbre de décision

In [16]:
# Création des noms des features et des classes
feature_names = ["Contient_gratuit", "Liens_3", "Expediteur_inconnu"]
class_names = label_encoders["Spam/Non-Spam"].inverse_transform([0,1])

# Exportation de l'arbre au format DOT
dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=feature_names,
                                class_names=class_names,
                                filled=True, rounded=True,
                                special_characters=True)

# Affichage de l'arbre
graph = graphviz.Source(dot_data)
graph

KeyError: 'Spam_Non_Spam'